# Модуль 06 - Объединение датафреймов
## Блок 02 - Метод merge

In [8]:
import pandas as pd
import numpy as np

genres = ['Drama', 'Action', 'Thriller']
ratings = pd.read_csv('module_05_ratings.csv')
movies = pd.read_csv('module_06_movies.csv')
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [7]:
# Считаем количество фильмов в жанре Fantasy.
movies['genres'].str.contains('Fantasy').sum()
# Сначала использовал count() балда... :) на первых порах всегда нужно узнать через info() 
# все параметры dataframe

654

In [9]:
joined = ratings.merge(movies, on='movieId', how='left')
joined.head()

,userId,movieId,rating,timestamp,title,genres
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama
1,1,1029,3.0,1260759179,Dumbo (1941),Animation|Children|Drama|Musical
2,1,1061,3.0,1260759182,Sleepers (1996),Thriller
3,1,1129,2.0,1260759185,Escape from New York (1981),Action|Adventure|Sci-Fi|Thriller
4,1,1172,4.0,1260759205,Cinema Paradiso (Nuovo cinema Paradiso) (1989),Drama


In [10]:
# проверка на отсутствие дубликатов после объединения dataframe's
len(ratings) == len(joined)

True

In [17]:
# Даны выгрузки статистики рекламных кампаний из Яндекс.Директа (файл “direct_stats.tsv”) 
# и заказов на сайте из CRM (файл “crm_stats.tsv”).

# В качестве разделителя используем \t
direct_stats = pd.read_csv('module_06_direct_stats.tsv', '\t')
crm_stats = pd.read_csv('module_06_crm_stats.tsv', '\t')
# Необходимо совместить эти выгрузки по дате и названию кампании. И посчитать стоимость 
# заказа для каждой кампании за каждый день. Т. е. отношение расходов (столбец cost) 
# к количеству заказов (столбец orders)

In [18]:
direct_stats.head()

,date,campaign,views,clicks,cost
0,2018-01-01,landings_promo,38120423,49557,1139801
1,2018-01-01,homepage_partner_1,5729483,12605,189073
2,2018-01-01,homepage_partner_2,4412029,9265,176040
3,2018-01-01,socdem_w_25-34_vip_test,913823,2559,89555
4,2018-01-02,landings_promo,40873806,61311,1471457


In [19]:
crm_stats.head()

,date,campaign,orders
0,2018-01-01,landings_promo,1487
1,2018-01-01,homepage_partner_1,386
2,2018-01-01,homepage_partner_2,315
3,2018-01-01,socdem_w_25-34_vip_test,85
4,2018-01-02,landings_promo,1605


In [21]:
direct_crm_joined = direct_stats.merge(crm_stats, on=['date','campaign'], how='left')
direct_crm_joined.head()

,date,campaign,views,clicks,cost,orders
0,2018-01-01,landings_promo,38120423,49557,1139801,1487
1,2018-01-01,homepage_partner_1,5729483,12605,189073,386
2,2018-01-01,homepage_partner_2,4412029,9265,176040,315
3,2018-01-01,socdem_w_25-34_vip_test,913823,2559,89555,85
4,2018-01-02,landings_promo,40873806,61311,1471457,1605


In [23]:
# проверка на дубликаты
len(direct_stats) == len(direct_crm_joined)

True

In [33]:
direct_crm_joined['order_cost'] = direct_crm_joined['cost'] / direct_crm_joined['orders']
direct_crm_joined.head()

,date,campaign,views,clicks,cost,orders,price,order_cost
0,2018-01-01,landings_promo,38120423,49557,1139801,1487,766.510424,766.510424
1,2018-01-01,homepage_partner_1,5729483,12605,189073,386,489.826425,489.826425
2,2018-01-01,homepage_partner_2,4412029,9265,176040,315,558.857143,558.857143
3,2018-01-01,socdem_w_25-34_vip_test,913823,2559,89555,85,1053.588235,1053.588235
4,2018-01-02,landings_promo,40873806,61311,1471457,1605,916.795639,916.795639


In [48]:
# Какое значение стоимости заказа получится к кампании landings_promo за 1 января 2018?
# Результат округлить до целого.
order_cost = direct_crm_joined[(direct_crm_joined['campaign'] == 'landings_promo') 
                   & (direct_crm_joined['date'] == '2018-01-01')]['order_cost'].sum()
# Использовал sum() для преобразования результата из dataframe в float64. полагаю это не лучший вариант.
round(order_cost)

767

766.5104236718224

In [50]:
# Возвращаемся к dataframe "joined"
# Напишем отдельную функцию для этой операции, а потом применим ее к каждой строке датафрейма joined 
# с помощью метода apply. Соответственно, логика функции должна быть такой:
# 1. Функция должна возвращать столько же столбцов, сколько элементов в списке genres.
# 2. Вместо названий жанров надо подставлять значение столбца rating, 
#    если жанр есть в столбце genres датафрейма.
# 3. Если названия жанра не оказалось, то возвращаем np.NaN

# используем list comprehension для умещения функции в одну строку
def genres_ratings(row):
    return pd.Series([row['rating'] if genre in row['genres'] else np.NAN for genre in genres])


In [62]:
# Применяем функцию к dataframe

joined[genres] = joined.apply(genres_ratings, axis = 1)
joined.head()

,userId,movieId,rating,timestamp,title,genres,Drama,Action,Thriller
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama,2.5,NaN,NaN
1,1,1029,3.0,1260759179,Dumbo (1941),Animation|Children|Drama|Musical,3.0,NaN,NaN
2,1,1061,3.0,1260759182,Sleepers (1996),Thriller,NaN,NaN,3.0
3,1,1129,2.0,1260759185,Escape from New York (1981),Action|Adventure|Sci-Fi|Thriller,NaN,2.0,2.0
4,1,1172,4.0,1260759205,Cinema Paradiso (Nuovo cinema Paradiso) (1989),Drama,4.0,NaN,NaN


In [54]:
# Если жанр встречается в текущей строке, то присвоить ему значение столбца rating. 
# Если жанра в строке нет, то ставим пустое значение np.NaN (т. е. указываем, что в ячейке нет числа).
#
# Здесь нам пригодится библиотека numpy (которую мы обозначили как np). 
# А пустое значение ставим, чтобы при подсчете среднего эти строки не учитывались в результате.
# Можно было ставить 0, но тогда это сильно исказит итоговый средний рейтинг жанра.

for genre in genres:
    print('Жанр {} средний рейтинг {:.2f}'.format(genre, joined[genre].mean()))

Жанр Drama средний рейтинг 3.68
Жанр Action средний рейтинг 3.45
Жанр Thriller средний рейтинг 3.52


In [71]:
# Необходимо установить, какие жанры фильмов имеют меньше всего оценок в наших данных.
# В качестве списка жанров используйте расширенную версию:
genres_extended = ['Drama', 'Action', 'Thriller', 'Comedy', 'Romance', 'War', 'Mystery', 'Crime']

def genres_ratings_extended(row):
    return pd.Series([row['rating'] if genre in row['genres'] 
                      else np.NAN for genre in genres_extended])
joined_extended = ratings.merge(movies, on='movieId', how='left')
joined_extended[genres_extended] = joined.apply(genres_ratings_extended, axis = 1)
joined_extended.head()

,userId,movieId,rating,timestamp,title,genres,Drama,Action,Thriller,Comedy,Romance,War,Mystery,Crime
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama,2.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1029,3.0,1260759179,Dumbo (1941),Animation|Children|Drama|Musical,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,1061,3.0,1260759182,Sleepers (1996),Thriller,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN
3,1,1129,2.0,1260759185,Escape from New York (1981),Action|Adventure|Sci-Fi|Thriller,NaN,2.0,2.0,NaN,NaN,NaN,NaN,NaN
4,1,1172,4.0,1260759205,Cinema Paradiso (Nuovo cinema Paradiso) (1989),Drama,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99999,671,6268,2.5,1065579370,Raising Victor Vargas (2002),Comedy|Drama|Romance,2.5,NaN,NaN,2.5,2.5,NaN,NaN,NaN
100000,671,6269,4.0,1065149201,Stevie (2002),Documentary,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100001,671,6365,4.0,1070940363,"Matrix Reloaded, The (2003)",Action|Adventure|Sci-Fi|Thriller|IMAX,NaN,4.0,4.0,NaN,NaN,NaN,NaN,NaN
100002,671,6385,2.5,1070979663,Whale Rider (2002),Drama,2.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [72]:
joined_extended.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100004 entries, 0 to 100003
Data columns (total 14 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100004 non-null  int64  
 1   movieId    100004 non-null  int64  
 2   rating     100004 non-null  float64
 3   timestamp  100004 non-null  int64  
 4   title      100004 non-null  object 
 5   genres     100004 non-null  object 
 6   Drama      44752 non-null   float64
 7   Action     27056 non-null   float64
 8   Thriller   25240 non-null   float64
 9   Comedy     38026 non-null   float64
 10  Romance    19336 non-null   float64
 11  War        5025 non-null    float64
 12  Mystery    7625 non-null    float64
 13  Crime      16266 non-null   float64
dtypes: float64(9), int64(3), object(2)
memory usage: 11.4+ MB


In [77]:
# Какое количество оценок имеет жанр с наименьшим количеством рейтингов?

# Вычисляем количество оценок по всем жанрам
joined_extended[genres_extended].count()

Drama       44752
Action      27056
Thriller    25240
Comedy      38026
Romance     19336
War          5025
Mystery      7625
Crime       16266
dtype: int64

In [73]:
# получаем жанр с минимальным количеством оценок. Можно было сделать и на предыдущем шаге, 
# но для наглядности сделал так.
joined_extended[genres_extended].count().min()

5025